In [ ]:
import os
import glob
import subprocess
import h5py
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
script_path = '/~/apply_elliptical_correction_polardatacube.py'

BEAMLINE = 'e02'
YEAR = '2025'
VISIT = 'mgXXXXX-X'
sub = 'subfolder_name'
au_calib_name = 'au_xgrating'
base_dir = f'/dls/{BEAMLINE}/data/{YEAR}/{VISIT}/processing/Merlin'
au_calib_dir = f'/dls/{BEAMLINE}/data/{YEAR}/{VISIT}/processing/Merlin/{au_calib_name}/' # The whole path can be manually specified

# (optional) Angle between the real space dimensions and the reciprocal space dimensions
R_Q_ROTATION = '0' 

also_rpl = 'False' # if 'True', the results will also be saved in '.rpl' format

mask_path = '' # if you would like to apply a certain mask to the diffraction patterns

fast_origin = True # if not 'True', the process includes the Bragg peak finding (the centre positions could be more accurate, but it needs more time)

In [ ]:
file_adrs = glob.glob(base_dir+'/'+sub+'/*/*/*_data.hdf5', recursive=True)
if file_adrs == []:
    file_adrs = glob.glob(base_dir+'/'+sub+'/*/*_data.hdf5', recursive=True)
    if file_adrs == []:
        file_adrs = glob.glob(base_dir+'/'+sub+'/*_data.hdf5', recursive=True)
        if file_adrs == []:
            print("Please make sure that the base directory and subfolder name are correct.")
            
print(len(file_adrs))
print(*file_adrs, sep='\n')

In [ ]:
data_labels = []
for adr in file_adrs:
    datetime = adr.split('/')[-2]
    if os.path.exists(os.path.dirname(adr) + "/" + datetime + "_azimuthal_data_centre.png"):
        continue
    else:
        data_labels.append(sub+'/'+adr.split('/')[-2])

print(len(data_labels))
print(*data_labels, sep='\n')

In [ ]:
code_path = base_dir + '/' + sub + '/cluster_logs'
if not os.path.exists(code_path):
    os.mkdir(code_path)
    
info_path = os.path.join(code_path, 'transformation_info.txt')

with open (info_path, 'w') as f:
    f.write(
        f"BEAMLINE = {BEAMLINE}\n"
        f"YEAR = {YEAR}\n"
        f"VISIT = {VISIT}\n"
        f"sub = {sub}\n"
        f"data_labels = {data_labels}\n"
        f"au_calib_dir = {au_calib_dir}\n"
        f"R_Q_ROTATION = {R_Q_ROTATION}\n"
        f"also_rpl = {also_rpl}\n"
        f"mask_path = {mask_path}\n"
        f"fast_origin = {fast_origin}\n"
            )

concurrent_jobs = 10 # number of slurm jobs running concurrently
bash_script_path = os.path.join(code_path, 'cluster_submit.sh')
with open(bash_script_path, 'w') as f:
    f.write("#!/usr/bin/env bash\n")
    f.write("#SBATCH --partition=cs04r\n")
    f.write("#SBATCH --job-name=rad_trans\n")
    f.write("#SBATCH --nodes=1\n")
    f.write("#SBATCH --ntasks-per-node=4\n")
    f.write("#SBATCH --cpus-per-task=1\n")
    f.write("#SBATCH --time=5:00:00\n")
    f.write("#SBATCH --mem=128G\n")
    f.write("#SBATCH --output=%s/%%j.out\n"%code_path)
    f.write("#SBATCH --error=%s/%%j.error\n\n"%code_path)
    f.write(f"#SBATCH --array=0-{len(data_labels)-1}%{concurrent_jobs}\n")
    
    f.write("module load python/epsic3.10\n")
    f.write(f'python {script_path} {info_path} $SLURM_ARRAY_TASK_ID\n')

sshProcess = subprocess.Popen(['ssh',
                               '-tt',
                               'wilson'],
                               stdin=subprocess.PIPE, 
                               stdout = subprocess.PIPE,
                               universal_newlines=True,
                               bufsize=0)
sshProcess.stdin.write("echo END\n")
sshProcess.stdin.write("sbatch "+bash_script_path+"\n")
sshProcess.stdin.write("uptime\n")
sshProcess.stdin.write("logout\n")
sshProcess.stdin.close()